##### Importing libraries

In [1]:
import sqlite3

import pandas as pd

from pprint import pprint

##### HTTP links of data

In [2]:
# Value = Source URL
dict_sources = {"produit" : "https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=0&single=true&output=csv",
                "magasins" : "https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=714623615&single=true&output=csv",
                "vente" : "https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=760830694&single=true&output=csv"}

pprint(dict_sources)

{'magasins': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=714623615&single=true&output=csv',
 'produit': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=0&single=true&output=csv',
 'vente': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=760830694&single=true&output=csv'}


##### Iterating over the sources to make dataframes and storing in a dictionary

In [3]:
# Empty dictionary to be populated in for loop below
dict_dataframes = {}

# iterate the dict sources 
for data_label, source in dict_sources.items():

    # print file_name and link with \n
    print(data_label,"\n",source)

    # read csv files
    data = pd.read_csv(source)

    # print the shape of each dataframe
    print(data.shape)
    
    # storing dataframe in the dictionary
    dict_dataframes[data_label] = data


produit 
 https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=0&single=true&output=csv
(5, 4)
magasins 
 https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=714623615&single=true&output=csv
(7, 3)
vente 
 https://docs.google.com/spreadsheets/d/e/2PACX-1vSawI56WBC64foMT9pKCiY594fBZk9Lyj8_bxfgmq-8ck_jw1Z49qDeMatCWqBxehEVoM6U1zdYx73V/pub?gid=760830694&single=true&output=csv
(30, 4)


##### Connecting SQL client

In [4]:
# database name. The database will be created in the same repo
database = "market_analysis"

# Connecting the database
client = sqlite3.connect(database=database)

##### Making tables in the database. The table names are produit, magasin and vente

In [5]:
# iterating on the dataframes dictionary and storing these dataframes as tables in sql database
for data_label, dataframe in dict_dataframes.items():

    # printing the table name and top 5 rows
    print(data_label,"\n",dataframe.head())

    # storing in the database
    dataframe.to_sql(name=data_label, con=client)

produit 
          Nom ID Référence produit   Prix  Stock
0  Produit A               REF001  49.99    100
1  Produit B               REF002  19.99     50
2  Produit C               REF003  29.99     75
3  Produit D               REF004  79.99    120
4  Produit E               REF005  39.99     80
magasins 
    ID Magasin      Ville  Nombre de salariés
0           1      Paris                  10
1           2  Marseille                   5
2           3       Lyon                   8
3           4   Bordeaux                  12
4           5      Lille                   6
vente 
          Date ID Référence produit  Quantité  ID Magasin
0  2023-05-27               REF001         5           1
1  2023-05-28               REF002         3           2
2  2023-05-29               REF003         2           1
3  2023-05-30               REF004         4           3
4  2023-05-31               REF005         7           2


##### Verifying if the data is readable

In [6]:
# iterating on the dict_dataframes keys to check if the shapes are same as in cell 3
for data_label in dict_dataframes.keys():

    # printing the table name
    print(data_label)

    # reading data from sql database
    df_read = pd.read_sql(sql="SELECT * FROM {}".format(data_label), con=client)

    # Looking the shape of df_read
    print(df_read.shape)

produit
(5, 5)
magasins
(7, 4)
vente
(30, 5)


In [7]:
# checking the extra column
df_read.head()

,index,Date,ID Référence produit,Quantité,ID Magasin
0,0,2023-05-27,REF001,5,1
1,1,2023-05-28,REF002,3,2
2,2,2023-05-29,REF003,2,1
3,3,2023-05-30,REF004,4,3
4,4,2023-05-31,REF005,7,2


Since the shapes are same, excluding the index, therefore, we can say that the data from the source URL are successfully inserted in the SQL database.